# Segmenting and Clustering Neighborhoods in Toronto
## Clustering and Visualization

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
import folium # map rendering library

After importing packages, we import the data processed from the previous notebook.

In [2]:
df = pd.read_csv("postcode_lat_lng.csv")
df

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M6C,York,Humewood-Cedarvale,43.693781,-79.428191
1,M6E,York,Caledonia-Fairbanks,43.689026,-79.453512
2,M9N,York,Weston,43.706876,-79.518188
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1S,Scarborough,Agincourt,43.794200,-79.262029
7,M1W,Scarborough,L'Amoreaux West,43.799525,-79.318389
8,M1X,Scarborough,Upper Rouge,43.836125,-79.205636
9,M3A,North York,Parkwoods,43.753259,-79.329656


We now have data that contain postal codes, borough names, latitudes and longitudes.
Next, we will extract the data that have "Toronto" in their borough names and cluster them later, since the purpose of this project is to cluster the neighborhoods in Toronto.

In [3]:
# create a list and save the boolean result of whether borough names include "Toronto"
# and use this list to select the subset of the data frame
isin_Tor = []
for borough in df[["Borough"]].iloc[:,0]:
    isin_Tor.append("Toronto" in borough)

df_tor = df[pd.Series(isin_Tor)]
df_tor

,Postcode,Borough,Neighborhood,Latitude,Longitude
29,M4E,East Toronto,The Beaches,43.676357,-79.293031
30,M4M,East Toronto,Studio District,43.659526,-79.340923
31,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558
32,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
33,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
34,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
35,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
36,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
37,M6G,Downtown Toronto,Christie,43.669542,-79.422564
38,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529


Now, we wil get the venues in or around those areas within a radius of 500 meters. <br/>
First, let's create the GET request URL.

In [11]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [13]:
toronto_venues = getNearbyVenues(names=df_tor["Neighborhood"],
                                latitudes=df_tor["Latitude"],
                                longitudes=df_tor["Longitude"])
print(toronto_venues)

The Beaches
Studio District
Business Reply Mail Processing Centre 969 Eastern
Harbourfront
Queen's Park
St. James Town
Berczy Park
Central Bay Street
Christie
Rosedale
Stn A PO Boxes 25 The Esplanade
Church and Wellesley
Lawrence Park
Roselawn
Davisville North
North Toronto West
Davisville
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
The Annex, North Midtown, Yorkville
Brockton, Exhibition Place, Parkdale Village
Adelaide, King, Richmond
Chinatown, Grange Park, Kensington Market
Harbourfront East, Toronto Islands, Union Station
Ryerson, Garden District
First Canadian Place, Underground city
The Beaches West, India Bazaar
The Danforth West, Riverdale
Parkdale, Roncesvalles
Little Portugal, Trinity
Harbord, University of Toronto
Commerce Court, Victoria Hotel
High Park, The Junction South
Runnymede, Swansea
Forest Hill North, Forest Hill West
Cabbagetown, St. Ja

Let's check the size of the resulting dataframe.

In [16]:
print(toronto_venues.shape)
toronto_venues.head()

(1719, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,Studio District,43.659526,-79.340923,Ed's Real Scoop,43.660656,-79.342019,Ice Cream Shop


Let's see how many venues were returned for each neighborhood.

In [17]:
toronto_venues.groupby("Neighborhood").count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,56,56,56,56,56,56
"Brockton, Exhibition Place, Parkdale Village",22,22,22,22,22,22
Business Reply Mail Processing Centre 969 Eastern,16,16,16,16,16,16
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",15,15,15,15,15,15
"Cabbagetown, St. James Town",47,47,47,47,47,47
Central Bay Street,86,86,86,86,86,86
"Chinatown, Grange Park, Kensington Market",91,91,91,91,91,91
Christie,17,17,17,17,17,17


### Next, we are going to analyze the data for each neighborhood.

In [25]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[["Venue Category"]], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot["Neighborhood"] = toronto_venues["Neighborhood"]

# move neighborhood column to the first column
cols_wo_neighbor = list(toronto_onehot.columns)
cols_wo_neighbor.remove("Neighborhood")

fixed_columns = ["Neighborhood"] + cols_wo_neighbor
toronto_onehot = toronto_onehot[fixed_columns]
toronto_onehot.head()

,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [26]:
toronto_onehot.shape

(1719, 234)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [28]:
toronto_grouped = toronto_onehot.groupby("Neighborhood").mean().reset_index()
toronto_grouped

,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.00,0.010000,0.000000,0.030000,0.000000,0.0000,0.000000,0.000000,0.000000,0.020000,0.000000,0.040000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.010000,0.020000,0.000000,0.000000,0.01,0.030000,0.010000,0.000000,0.000000,0.050000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.080000,0.000000,0.00000,0.00000,0.000000,0.00,0.01,0.000000,0.000000,0.020000,0.000000,0.020000,0.00000,0.000000,0.000000,0.000000,0.000000,0.010000,0.010000,0.0000

In [29]:
toronto_grouped.shape

(39, 234)

Let's get each neighborhood along with the top 5 most common venues in that area and put them into a dataframe. <br/>
We start with defining a function to sort the venues in descending order.

In [27]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [31]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Bar,Steakhouse,Thai Restaurant,Burger Joint,Sushi Restaurant,Asian Restaurant,Breakfast Spot,Pizza Place
1,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Cheese Shop,Café,Seafood Restaurant,Beer Bar,Steakhouse,Farmers Market,Creperie
2,"Brockton, Exhibition Place, Parkdale Village",Café,Coffee Shop,Breakfast Spot,Pet Store,Bakery,Nightclub,Performing Arts Venue,Climbing Gym,Restaurant,Burrito Place
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Auto Workshop,Comic Shop,Pizza Place,Restaurant,Burrito Place,Skate Park,Smoke Shop,Brewery,Spa
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Lounge,Airport Service,Airport Terminal,Airport,Bar,Coffee Shop,Sculpture Garden,Boat or Ferry,Boutique,Airport Gate


## Cluster Neighborhoods

First, we try to run k-means to cluster the neighborhood into 5 clusters.

In [69]:
# set number of clusters
k=5

# drop the Neighborhood column
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=k, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [70]:
neighborhoods_venues_sorted.drop("Cluster Labels", 1, inplace=True)

In [71]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_tor

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
29,M4E,East Toronto,The Beaches,43.676357,-79.293031,4,Trail,Health Food Store,Pub,Yoga Studio,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
30,M4M,East Toronto,Studio District,43.659526,-79.340923,4,Café,Coffee Shop,American Restaurant,Bakery,Italian Restaurant,Gastropub,Yoga Studio,Fish Market,Sandwich Place,Pet Store
31,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,4,Light Rail Station,Auto Workshop,Comic Shop,Pizza Place,Restaurant,Burrito Place,Skate Park,Smoke Shop,Brewery,Spa
32,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,4,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Café,Mexican Restaurant,Gym / Fitness Center,Brewery,Shoe Store
33,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494,4,Coffee Shop,Gym,Park,Yoga Studio,College Cafeteria,Smoothie Shop,Burger Joint,Burrito Place,Sandwich Place,Café
34,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,4,Café,Coffee Shop,Restaurant,Hotel,Breakfast Spot,Clothing Store,Cocktail Bar,Beer Bar,Cosmetics Shop,Italian Restaurant
35,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,4,Coffee Shop,Cocktail Bar,Bakery,Cheese Shop,Café,Seafood Restaurant,Beer Bar,Steakhouse,Farmers Market,Creperie
36,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,4,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Burger Joint,Ice Cream Shop,Salad Place,Middle Eastern Restaurant,Juice Bar,Japanese Restaurant
37,M6G,Downtown Toronto,Christie,43.669542,-79.422564,1,Grocery Store,Café,Park,Coffee Shop,Restaurant,Convenience Store,Italian Restaurant,Candy Store,Baby Store,Nightclub
38,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,2,Park,Playground,Trail,Yoga Studio,Dessert Shop,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


Finally, let's visualize the resulting clusters. <br/>
Firstm let's get the grographical coordinates of Toronto.

In [63]:
loc_name = "Toronto, Canada"
geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(loc_name)
latitude = location.latitude
longitude = location.longitude
print("The goegraphical coordinate of Toronto is {}, {}.".format(latitude, longitude))

The goegraphical coordinate of Toronto is 43.653963, -79.387207.


In [72]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Now, we have 5 clusters generated by the K-means algorithm. Let's examine them one by one.

### Cluster 1

In [73]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, 
                     toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
96,Central Toronto,0,Jewelry Store,Trail,Sushi Restaurant,Bus Line,Yoga Studio,Diner,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store


This cluster only has one point, which is the red one in the map. <br/>
Compared to the venues in other clusters, the most special part of this cluster is that Jewelry store is the first common venue, which is not included in the top 10 common venues of any other clusters.

### Cluster 2

In [74]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, 
                     toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,Downtown Toronto,1,Grocery Store,Café,Park,Coffee Shop,Restaurant,Convenience Store,Italian Restaurant,Candy Store,Baby Store,Nightclub
60,Central Toronto,1,Café,Sandwich Place,Coffee Shop,Pub,History Museum,Burger Joint,Indian Restaurant,Flower Shop,Middle Eastern Restaurant,BBQ Joint
64,West Toronto,1,Café,Coffee Shop,Breakfast Spot,Pet Store,Bakery,Nightclub,Performing Arts Venue,Climbing Gym,Restaurant,Burrito Place
88,Downtown Toronto,1,Café,Bookstore,Sandwich Place,Bar,Italian Restaurant,Japanese Restaurant,Restaurant,Bakery,Sushi Restaurant,Beer Bar
92,West Toronto,1,Thai Restaurant,Café,Mexican Restaurant,Bar,Bookstore,Fast Food Restaurant,Speakeasy,Cajun / Creole Restaurant,Italian Restaurant,Flea Market


We can see Café is in the top 2 common venues of every area in this cluster; obviously, that's why they were clustered to the same group.

### Cluster 3

In [75]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, 
                     toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
38,Downtown Toronto,2,Park,Playground,Trail,Yoga Studio,Dessert Shop,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
41,Central Toronto,2,Park,Swim School,Bus Line,Yoga Studio,Diner,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant
98,Central Toronto,2,Tennis Court,Park,Restaurant,Department Store,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


This cluster has more parks and places for sports. Parks and Yoga Studios appear more than 1 time in this cluster.

### Cluster 4

In [76]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, 
                     toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
42,Central Toronto,3,Garden,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


This cluster is the light green point in the map. Garden is its first most common venue, and it does not appear in any other clusters' top 10 common venues.

### Cluster 5

In [77]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, 
                     toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
29,East Toronto,4,Trail,Health Food Store,Pub,Yoga Studio,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
30,East Toronto,4,Café,Coffee Shop,American Restaurant,Bakery,Italian Restaurant,Gastropub,Yoga Studio,Fish Market,Sandwich Place,Pet Store
31,East Toronto,4,Light Rail Station,Auto Workshop,Comic Shop,Pizza Place,Restaurant,Burrito Place,Skate Park,Smoke Shop,Brewery,Spa
32,Downtown Toronto,4,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Café,Mexican Restaurant,Gym / Fitness Center,Brewery,Shoe Store
33,Downtown Toronto,4,Coffee Shop,Gym,Park,Yoga Studio,College Cafeteria,Smoothie Shop,Burger Joint,Burrito Place,Sandwich Place,Café
34,Downtown Toronto,4,Café,Coffee Shop,Restaurant,Hotel,Breakfast Spot,Clothing Store,Cocktail Bar,Beer Bar,Cosmetics Shop,Italian Restaurant
35,Downtown Toronto,4,Coffee Shop,Cocktail Bar,Bakery,Cheese Shop,Café,Seafood Restaurant,Beer Bar,Steakhouse,Farmers Market,Creperie
36,Downtown Toronto,4,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Burger Joint,Ice Cream Shop,Salad Place,Middle Eastern Restaurant,Juice Bar,Japanese Restaurant
39,Downtown Toronto,4,Coffee Shop,Restaurant,Café,Cocktail Bar,Seafood Restaurant,Hotel,Beer Bar,Japanese Restaurant,Creperie,Farmers Market
40,Downtown Toronto,4,Coffee Shop,Gay Bar,Japanese Restaurant,Sushi Restaurant,Restaurant,Yoga Studio,Pub,Mediterranean Restaurant,Café,Hotel


There are multiple kinds of venues appear many times in this cluster, such Coffee Shop, Café, Pub, Restaurant, Bakery, etc. <br/>
So, we can know that this cluster is mixed with many kinds of venues, and it is also the biggest cluster.